# Availability of Sources
There seemed to be certain regions, which are not covered by Landsat 8 Tier 1 Level-2 images, that yield in empty collections. To dispel all doubts about this, availability should be studied systematically.

## Filter Samples by Country

In [1]:
import pandas as pd
import os

PROJECT_ROOT = '/'.join(os.getcwd().split('/')[:-2])
DATA_DIR = os.path.join(PROJECT_ROOT, "data")

metadata_path = os.path.join(DATA_DIR, "fmow_v1.1/rgb_metadata.csv")
df = pd.read_csv(metadata_path)

mask = df['country_code'].astype(str) == "PRK"
filtered_PKR = df[mask]
num_samples_PRK = len(filtered_PKR)
print(f"{num_samples_PRK} samples are from North Korea.")

1938 samples are from North Korea.


## Check Source Availability for PRK

In [2]:
import ee
import geemap

EE_PROJECT_NAME = 'seeing-the-big-picture'

try:
    ee.Authenticate()
    ee.Initialize(project=EE_PROJECT_NAME)
except Exception as e:
    print("Please authenticate Earth Engine: earthengine authenticate")
    raise

In [3]:
from tqdm.notebook import tqdm

count_na = 0
for sample_PKR in tqdm(zip(filtered_PKR['lat'], filtered_PKR['lon'])):
    lat, lon = sample_PKR

    bbox = [lon, lat,
        lon + 0.2, lat + 0.2]

    region = ee.Geometry.Rectangle(bbox)
    
    col = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
           .filterBounds(region))
    
    def scale_l8(image):
        return (image
                .select(['SR_B2', 'SR_B3', 'SR_B4'])
                .multiply(0.0000275)
                .add(-0.2))
    
    scol = col.map(scale_l8)
    
    if scol.size().getInfo() == 0:
        count_na += 1
        print(f"No sources available for: {lat, lon}")

print(f"{count_na} of {num_samples_PRK} samples in north korea have no available Landsat 8 Tier 1 Level-2 images.")

0it [00:00, ?it/s]

0 of 1938 samples in north korea have no available Landsat 8 Tier 1 Level-2 images.


That is **all regions** of interest in **north korea** can be backed with **context data**.